EXPOSURE: Maternal Decision-Making Autonomy

OUTCOME: Mode of Delivery

Variable Definitions:

* LTMID - Listening to Mothers ID number
* Q610 - In all how many babies have you had? Please include your new baby. (inclusion criteria - first pregnancies)
* Q885 - During your pregnancy did you ask your maternity care provider to schedule a c-section before the start of labor (before regular contractions)? (do we need to account for "intent" to have a c-section?)
* Q1250 Was the c-section planned ahead of time that is was the decision to have a c-section made before you went into labor? (inclusion criteria)
* Q1255 - What was the main reason for your recent c-section? (Primary c-section)
* insurance - Insurance during pregnancy (demographic)
* racegrp - race/ethnicity
* Q1860 - What is the highest level of education you have completed or the highest degree you have received?
* inccat - Income as a categorical percent of Federal Poverty Guideline in 2016 (FPG)taxes?

**Maternal Autonomy**

* Q1325a - How much do you agree with the following statements about your recent experience of labor and birth? The delivery room staff encouraged me to make decisions about how I wanted my birth to progress.

**Mode of Delivery**

* Q1115 - When you recently gave birth was your baby born vaginally or by c-section? (outcome)


In [31]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import logit, probit, poisson, ols

In [5]:
filepath = r"C:\Users\Nigel\Git\perinatal_epi\Data\LTMdata.Public VersionFINAL.csv"

df = pd.read_csv(filepath)
df.head()

,LTMID,mode,Q_Language,Q510a,Q520,Q610,Q705,Q715,Q716,Q740,...,workagef,height,wtgain,WtBeforePreg,bmi,currentwt,fampctpov,inccat,sampwt,statewt
0,70001n,1,EN,1.0,1,2,1.0,5.0,NaN,3.0,...,NaN,60.0,0.0,198.0,38.67,187.0,74.074074,1.0,19.950387,65.930595
1,70002m,1,EN,2.0,1,2,1.0,3.0,2.0,3.0,...,26.0,68.0,23.0,150.0,22.81,145.0,425.925926,5.0,30.288510,100.095275
2,70003n,1,ES,3.0,2,4,1.0,1.0,1.0,1.0,...,NaN,60.0,10.0,148.0,28.90,154.0,24.554942,1.0,23.204468,76.684446
3,70006c,2,EN,1.0,2,4,2.0,4.0,NaN,1.0,...,NaN,64.0,25.0,90.0,15.45,120.0,63.291139,1.0,21.639044,71.511148
4,70007s,3,EN,3.0,1,4,1.0,4.0,NaN,3.0,...,NaN,65.0,16.0,182.0,30.29,189.0,191.835482,2.0,20.981247,69.337307


In [6]:
cols = ['LTMID','Q1115', 'Q1250','Q1325a']
# mode of delivery, unplanned v. planned c-section, autonomy proxy

sample_df = df[cols].rename(columns={'Q1115':'MoD','Q1325a':'autonomy'})

In [7]:
# autonomy freq
sample_df.groupby(['autonomy'])['LTMID'].count()

autonomy
1.0    1069
2.0     492
3.0     304
4.0     106
5.0      96
Name: LTMID, dtype: int64

In [8]:
# planned v. unplanned freq
sample_df.groupby(['Q1250'])['LTMID'].count()

Q1250
1.0    406
2.0    320
Name: LTMID, dtype: int64

In [9]:
# mode of delivery freq
sample_df.groupby(['MoD'])['LTMID'].count()

MoD
1.0    1799
2.0     730
Name: LTMID, dtype: int64

In [10]:
sample_df.isnull().sum(axis=0)

LTMID          0
MoD           10
Q1250       1813
autonomy     472
dtype: int64

In [11]:
len(sample_df)

2539

In [12]:
# drop missing values for exposure and explanatory variables
sample_df = sample_df.dropna(subset=['MoD','autonomy'])
sample_df.head(20)

,LTMID,MoD,Q1250,autonomy
0,70001n,1.0,NaN,1.0
1,70002m,1.0,NaN,1.0
2,70003n,1.0,NaN,1.0
3,70006c,1.0,NaN,5.0
4,70007s,1.0,NaN,2.0
5,70008m,1.0,NaN,1.0
6,70010x,1.0,NaN,3.0
7,70013m,1.0,NaN,1.0
8,70016u,1.0,NaN,3.0
9,70017r,1.0,NaN,2.0


In [13]:
len(sample_df)

2067

In [14]:
# create combined column for autonomy (strongly agree and agree + all others)
sample_df['autonomy_'] = np.where(((sample_df['autonomy'] == 1) | (sample_df['autonomy'] == 2)), 1, 0)

In [15]:
len(sample_df)

2067

In [16]:
sample_df.head(20)

,LTMID,MoD,Q1250,autonomy,autonomy_
0,70001n,1.0,NaN,1.0,1
1,70002m,1.0,NaN,1.0,1
2,70003n,1.0,NaN,1.0,1
3,70006c,1.0,NaN,5.0,0
4,70007s,1.0,NaN,2.0,1
5,70008m,1.0,NaN,1.0,1
6,70010x,1.0,NaN,3.0,0
7,70013m,1.0,NaN,1.0,1
8,70016u,1.0,NaN,3.0,0
9,70017r,1.0,NaN,2.0,1


In [17]:
len(sample_df)

2067

In [18]:
sample_df.groupby(['MoD','Q1250'])['LTMID'].count()

MoD  Q1250
2.0  1.0       61
     2.0      247
Name: LTMID, dtype: int64

In [19]:
# remove planned cesearan sections from the sample
exclusion_criteria = (sample_df['Q1250'] == 1.0)

In [20]:
# autonomy among those that planned their cesarean section -- interesting
sample_df[exclusion_criteria].groupby(['autonomy_'])['LTMID'].count()

autonomy_
0    19
1    42
Name: LTMID, dtype: int64

In [21]:
# remove those who planned to have a c-section and simplify the dataset for the first model
sample_df = sample_df[~exclusion_criteria][['autonomy_','MoD']]

In [22]:
# recode the response variable to be 0 = vaginal delivery, 1 = cesarean delivery
sample_df['MoD_'] = np.where(sample_df['MoD'] == 1.0, 0, 1)

In [23]:
sample_df.head()

,autonomy_,MoD,MoD_
0,1,1.0,0
1,1,1.0,0
2,1,1.0,0
3,0,1.0,0
4,1,1.0,0


In [33]:
# build the logistic regression model
#logit_mod = sm.Logit(sample_df.MoD_, sample_df.autonomy_)

logit_mod = logit("MoD_ ~ autonomy_",sample_df).fit()

#logit_res = logit_mod.fit(disp=0)
#print(logit_res.summary())
logit_mod.summary()

Optimization terminated successfully.
         Current function value: 0.372954
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                   MoD_   No. Observations:                 2006
Model:                          Logit   Df Residuals:                     2004
Method:                           MLE   Df Model:                            1
Date:                Sun, 11 Dec 2022   Pseudo R-squ.:                0.003048
Time:                        22:45:04   Log-Likelihood:                -748.15
converged:                       True   LL-Null:                       -750.43
Covariance Type:            nonrobust   LLR p-value:                   0.03245
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.7194      0.126    -13.621      0.000      -1.967      -1.472
autonomy_     -0.3257      0.150     -2.175      0.030      -0.619      -0.032
==============================================================================
"""

In [34]:
# calculate the odds ratios

params = logit_mod.params
conf = logit_mod.conf_int()
conf['Odds Ratio'] = params
conf.columns = ['5%', '95%', 'Odds Ratio']
print(np.exp(conf))

                 5%       95%  Odds Ratio
Intercept  0.139905  0.229473    0.179177
autonomy_  0.538365  0.968309    0.722013
